# Dask Implementation

In [1]:
import numpy as np
import dask
import dask.dataframe as dd
from dask.distributed import Client
import joblib
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import LinearSVC
import time

SEED = 42

In [2]:
client = Client('172.31.88.97:8786')
client = client.restart()
print(client)

<Client: 'tcp://172.31.88.97:8786' processes=0 threads=0, memory=0 B>


## Read data

In [3]:
full_data = dd.read_csv('/home/ubuntu/data/cleaned_data.csv', sample=2**30, assume_missing=False)

In [7]:
print(len(full_data.index))

39435236


In [8]:
full_data.head(5)

,legId,searchDate,flightDate,destinationAirport,baseFare,seatsRemaining,totalTravelDistance,durationSeconds,dateDelta
0,222cfd6d1b0d5732602a3e82ad7730c3,0,45,BOS,65.48,4,947,17400,45
1,71cf5163f5efbd007c87aeef85e0c2cc,0,45,BOS,161.86,9,947,9420,45
2,141ef83862caac6be402158433b55c1f,0,45,BOS,161.86,2,947,9660,45
3,5921ef14d28a822fc25eb9d7879134a9,0,45,BOS,158.14,3,956,21780,45
4,3ce43c14cd65f1d4303a4b90093f328c,0,45,BOS,151.63,9,947,29580,45


In [ ]:
X_reg_train = pd.read_csv("")
X_reg_test = pd.read_csv("")
X_class_train = pd.read_csv("")
X_class_test = pd.read_csv("")

y_reg_train = pd.read_csv("")
y_reg_test = pd.read_csv("")
y_class_train = pd.read_csv("")
y_class_test = pd.read_csv("")

In [4]:
def dask_time_algorithm(subset_prop, algorithm_name, SEED=SEED): #algorithm_name one of 'KNN', 'RF', 'KMEANS'
    if algorithm_name.isin(["KNN", "RF"]):
        X_reg_train_subset = X_reg_train.head(int(len(X_reg_train) * subset_prop))
        X_reg_test_subset = X_reg_test.head(int(len(X_reg_test) * subset_prop))
        y_reg_train_subset = X_reg_test.head(int(len(X_reg_test) * subset_prop))
        y_reg_test_subset = X_reg_test.head(int(len(X_reg_test) * subset_prop))
        
    elif algorithm_name.isin(["KMEANS"]):
        X_class_train_subset = X_class_train.head(int(len(X_class_train) * subset_prop))
        X_class_test_subset = X_class_test.head(int(len(X_class_test) * subset_prop))
        y_class_train_subset = X_class_test.head(int(len(X_class_test) * subset_prop))
        y_class_test_subset = X_class_test.head(int(len(X_class_test) * subset_prop))
        
    start_time = time.time()
        
    if algorithm_name == "KNN":
        with joblib.parallel_backend('dask'):
            knn_model = KNeighborsRegressor(n_neighbors=5)
            knn_model.fit(X_reg_train_subset, y_reg_train_subset)
            metric = knn_model.score(X_reg_test_subset, y_reg_test_subset) # metric is KNN accuracy
                        
    elif algorithm_name == "KMEANS":
        with joblib.parallel_backend('dask'):
            kmeans_model = KMeans(n_clusters=2, random_state=SEED, n_init="auto")
            kmeans_model.fit(X_reg_train_subset)
            metric = None # No performance metric for KMEANS (unsupervised)
            
    elif algorithm_name == "RF":
        with joblib.parallel_backend('dask'):
            rf_model = RandomForestRegressor(max_depth=2, random_state=SEED)
            rf_model.fit(X_reg_train_subset, y_reg_train_subset)
            y_pred = rf_model.predict(X_test_reg_subset)
            metric = mean_squared_error(y_test_reg_subset, y_pred) # metric is random forest MSE
            
    end_time = time.time()
    time_delta = end_time - start_time
    return time_delta, metric

NameError: name 'SEED' is not defined

In [ ]:
# Example implementation for KNN

props = [.1, .3, .5, .7, 1]
times = []
metrics = []

for prop in props:
    time, metric = dask_time_algorithm(prop, "KNN")
    times.append(time)
    metrics.append(metric)
    
print(times)
print(metrics)